In [1]:
from selenium import webdriver
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException

driver=webdriver.Edge()

url='https://esf.fang.com/house-a015277-b02315/'
driver.get(url)

In [2]:
string_list=[]

i=0

while i<20:
    table =driver.find_element(By.CSS_SELECTOR, 'div.shop_list.shop_list_4')
    # Extract table houses
    houses = table.find_elements(By.TAG_NAME, 'dl')

    # Initialize lists to store data
    data = []

    # Loop through rows and extract data
    for house in houses:
        title = house.find_element(By.CSS_SELECTOR, "span.tit_shop").text
        price = house.find_element(By.CSS_SELECTOR, "span.red").text
        details = house.find_element(By.CSS_SELECTOR, "p.tel_shop").text
        data.append({"title": title, "price": price, "details": details})

    df = pd.DataFrame(data)
    string_list.append(df)
    #driver.refresh()
    try:
        next_page=driver.find_element('class name','last')
        next_page.click()
        i+=1
    except NoSuchElementException:
        break

In [3]:
driver.quit()

In [4]:
df_full=pd.concat(string_list)
df_full

,title,price,details
0,"海淀现房无中阶费,楼层可选,學院路平层大三居,房源有限尽快",1450万,3室2厅 | 180㎡ |高层 （共26层） | 南北向 | 2022年建 |白海涛
1,总监推荐 万橡府中间排 前后花园 诚意出售 价格可谈,2350万,3室2厅 | 185.05㎡ |高层 （共16层） | 南北向 | 2014年建 |冯磊
2,"新上,老客户房源,橡树湾二期双卫三居可做四居,随时可看",1470万,3室1厅 | 143.66㎡ |中层 （共18层） | 南北向 | 2010年建 |霍远飞
3,四期华润精装 保持如新 看房随时 诚意出售 采光无遮挡,1600万,3室2厅 | 139㎡ |低层 （共29层） | 南向 | 2014年建 |冯磊
4,"(豪装下跃,中间位置)功能区明确,使用空间大,别墅体验",3999万,4室3厅 | 553.91㎡ | 底层 （共4层） | 南北向 | 2019年建 |林新雨
...,...,...,...
55,南北通透华润橡树湾5室3厅精装修,2300万,5室3厅 | 281.7㎡ | 底层 （共10层） | 南北向 | 2010年建 |霍远飞
56,南北通透华润橡树湾3室2厅精装修,1660万,3室2厅 | 154.6㎡ |中层 （共21层） | 南北向 | 2011年建 |王福兴
57,出售 华润橡树湾 3室 南北向 精装修,2099万,3室2厅 | 178㎡ |高层 （共29层） | 南北向 | 2023年建 |王现伟
58,4室2厅世华龙樾四期,4500万,4室2厅 | 331.18㎡ |中层 （共5层） | 东南北向 | 2019年建 |杨喜武


In [5]:
df_full['area'] = df_full['details'].str.extract(r'(\d+\.?\d*)\s*㎡').astype(float)
df_full['price'] = df_full['price'].str.replace('万', '').astype(float) * 10000

In [6]:
df_full

,title,price,details,area
0,"海淀现房无中阶费,楼层可选,學院路平层大三居,房源有限尽快",14500000.0,3室2厅 | 180㎡ |高层 （共26层） | 南北向 | 2022年建 |白海涛,180.00
1,总监推荐 万橡府中间排 前后花园 诚意出售 价格可谈,23500000.0,3室2厅 | 185.05㎡ |高层 （共16层） | 南北向 | 2014年建 |冯磊,185.05
2,"新上,老客户房源,橡树湾二期双卫三居可做四居,随时可看",14700000.0,3室1厅 | 143.66㎡ |中层 （共18层） | 南北向 | 2010年建 |霍远飞,143.66
3,四期华润精装 保持如新 看房随时 诚意出售 采光无遮挡,16000000.0,3室2厅 | 139㎡ |低层 （共29层） | 南向 | 2014年建 |冯磊,139.00
4,"(豪装下跃,中间位置)功能区明确,使用空间大,别墅体验",39990000.0,4室3厅 | 553.91㎡ | 底层 （共4层） | 南北向 | 2019年建 |林新雨,553.91
...,...,...,...,...
55,南北通透华润橡树湾5室3厅精装修,23000000.0,5室3厅 | 281.7㎡ | 底层 （共10层） | 南北向 | 2010年建 |霍远飞,281.70
56,南北通透华润橡树湾3室2厅精装修,16600000.0,3室2厅 | 154.6㎡ |中层 （共21层） | 南北向 | 2011年建 |王福兴,154.60
57,出售 华润橡树湾 3室 南北向 精装修,20990000.0,3室2厅 | 178㎡ |高层 （共29层） | 南北向 | 2023年建 |王现伟,178.00
58,4室2厅世华龙樾四期,45000000.0,4室2厅 | 331.18㎡ |中层 （共5层） | 东南北向 | 2019年建 |杨喜武,331.18


In [7]:
df_full[['price', 'area']].to_csv(
    'house_prices_area.csv',  # 文件名
    index=False,              # 不保存索引
    encoding='utf-8-sig'      # 支持中文的编码格式
)

saved_df = pd.read_csv('house_prices_area.csv')